In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [12]:
from DataParser import DataParser
import numpy as np
dataSet = DataParser("../data/DataForML.mat")
x_train, y_train = dataSet.get_train_multipath(True)
x_test, y_test = dataSet.get_test_multipath(True)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

def flatten_data(x, y):
    x = np.reshape(x, (3, -1))
    x = np.stack([np.real(x), np.imag(x)], axis=0)
    x = np.reshape(x, (6, -1))
    x = np.transpose(x)
    y = np.reshape(y, (-1))
    return x, y

x_train,y_train = flatten_data(x_train, y_train)
x_test, y_test = flatten_data(x_test, y_test)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(y_test)

(3, 8, 20000) (8, 20000)
(3, 8, 25000) (8, 25000)
(160000, 6) (160000,)
(200000, 6) (200000,)
[ 0.70710678+0.70710678j -0.70710678-0.70710678j -0.70710678+0.70710678j
 ... -0.70710678+0.70710678j  0.70710678-0.70710678j
  0.70710678-0.70710678j]


In [3]:
from sklearn.linear_model import LinearRegression

In [13]:
reg_real = LinearRegression().fit(x_train, y_train.real)
reg_imag = LinearRegression().fit(x_train, y_train.imag)

In [14]:
# predict
y_pred_real = reg_real.predict(x_test)
y_pred_imag = reg_imag.predict(x_test)
y_pred = y_pred_real + 1j * y_pred_imag

def get_quadrant(x: np.ndarray):
    x_real = x.real.copy()
    x_imag = x.imag.copy()
    x_real[x_real >= 0] = 1
    x_real[x_real < 0] = -1
    x_imag[x_imag >= 0] = 1
    x_imag[x_imag < 0] = -1
    # encode with quadrant number
    x_real = x_real + 1
    x_imag = x_imag + 1
    x_real = x_real.astype(int)
    x_imag = x_imag.astype(int)
    return x_real * 3 + x_imag

y_pred_quadrant = get_quadrant(y_pred)
y_test_quadrant = get_quadrant(y_test)

print(y_pred.shape, y_test.shape)


(200000,) (200000,)


In [6]:
# calculate accuracy
def get_accuracy(y_pred, y_test):
    return np.sum(y_pred == y_test) / len(y_test)

print(get_accuracy(y_pred_quadrant, y_test_quadrant))

0.944755


In [7]:
from sklearn import svm;

In [8]:
def train(train_data, train_labels):
    train_data = train_data.flatten()
    train_labels = train_labels.flatten()

    train_data = np.stack([train_data.real, train_data.imag], axis=-1)

    train_labels_real = train_labels.real > 0
    train_labels_imag = train_labels.imag > 0
    
    #Create a svm Classifier
    clf_real = svm.SVC(kernel='linear')
    clf_imag = svm.SVC(kernel='linear') 

    #Train the model using the training sets
    clf_real.fit(train_data, train_labels_real)
    clf_imag.fit(train_data, train_labels_imag)

    return (clf_real, clf_imag)
def predict(test_data, labels_data, model):
    clf_real = model[0]
    clf_imag = model[1]
    test_data = test_data.flatten()

    test_data = np.stack([test_data.real, test_data.imag], axis=-1)

    test_labels_real = labels_data.real > 0
    test_labels_imag = labels_data.imag > 0

    labels_pred_real = clf_real.predict(test_data)
    labels_pred_imag = clf_imag.predict(test_data)

    labels_pred = np.where(labels_pred_real & labels_pred_imag, 1, np.where(~labels_pred_real & labels_pred_imag, 2, np.where(~labels_pred_real & ~labels_pred_imag, 3,np.where(labels_pred_real & ~labels_pred_imag, 4, 0))))

    test_labels = np.where(test_labels_real & test_labels_imag, 1,np.where(~test_labels_real & test_labels_imag, 2,np.where(~test_labels_real & ~test_labels_imag, 3,np.where(test_labels_real & ~test_labels_imag, 4, 0))))
    labels_pred = labels_pred.reshape((8, 25000))
    test_labels = test_labels.reshape((8, 25000))

    return labels_pred, test_labels

In [15]:
print(y_pred, y_test)

[ 1.45475023+0.41785817j -1.18007838+0.02250308j -0.48256452+0.05962413j
 ... -0.6492475 +0.83429801j  0.47627417-0.98444107j
  0.54925657-0.16463101j] [ 0.70710678+0.70710678j -0.70710678-0.70710678j -0.70710678+0.70710678j
 ... -0.70710678+0.70710678j  0.70710678-0.70710678j
  0.70710678-0.70710678j]


In [16]:
model = train(x_train, y_train)
labels_pred, test_labels = predict(x_test, y_test, model)

ValueError: Found input variables with inconsistent numbers of samples: [960000, 160000]